In [2]:
# Dependencies
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

In [3]:
#target city
target_city = "San Diego, CA"

In [4]:
#endpoint url

target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)

In [5]:
# Run a request to endpoint and convert result to json
sd_data = requests.get(target_url).json()

# Print the json
print(sd_data)

{'results': [{'address_components': [{'long_name': 'San Diego', 'short_name': 'San Diego', 'types': ['locality', 'political']}, {'long_name': 'San Diego County', 'short_name': 'San Diego County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'San Diego, CA, USA', 'geometry': {'bounds': {'northeast': {'lat': 33.114249, 'lng': -116.90816}, 'southwest': {'lat': 32.534856, 'lng': -117.3097969}}, 'location': {'lat': 32.715738, 'lng': -117.1610838}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 33.114249, 'lng': -116.90816}, 'southwest': {'lat': 32.534856, 'lng': -117.3097969}}}, 'place_id': 'ChIJSx6SrQ9T2YARed8V_f0hOg0', 'types': ['locality', 'political']}], 'status': 'OK'}


In [6]:
# Print the json (pretty printed)
print(json.dumps(sd_data, indent=2, sort_keys=True))

{
  "results": [
    {
      "address_components": [
        {
          "long_name": "San Diego",
          "short_name": "San Diego",
          "types": [
            "locality",
            "political"
          ]
        },
        {
          "long_name": "San Diego County",
          "short_name": "San Diego County",
          "types": [
            "administrative_area_level_2",
            "political"
          ]
        },
        {
          "long_name": "California",
          "short_name": "CA",
          "types": [
            "administrative_area_level_1",
            "political"
          ]
        },
        {
          "long_name": "United States",
          "short_name": "US",
          "types": [
            "country",
            "political"
          ]
        }
      ],
      "formatted_address": "San Diego, CA, USA",
      "geometry": {
        "bounds": {
          "northeast": {
            "lat": 33.114249,
            "lng": -116.90816
          },
          

In [7]:
#extract lat/long
# Extract latitude and longitude
lat = sd_data["results"][0]["geometry"]["location"]["lat"]
lng = sd_data["results"][0]["geometry"]["location"]["lng"]

# Print the latitude and longitude
print('''
    City: {0}
    Latitude: {1}
    Longitude: {2}
    '''.format(target_city, lat, lng))


    City: San Diego, CA
    Latitude: 32.715738
    Longitude: -117.1610838
    


In [8]:
#bring in a csv that includes zips for city
unsorted_df = pd.read_csv("city_zip.csv")
unsorted_df.head()

,Unnamed: 0,zip,City,Metro,CountyName
0,117,91910,Chula Vista,San Diego-Carlsbad,San Diego
1,176,92126,San Diego,San Diego-Carlsbad,San Diego
2,197,91911,Chula Vista,San Diego-Carlsbad,San Diego
3,221,92109,San Diego,San Diego-Carlsbad,San Diego
4,224,92101,San Diego,San Diego-Carlsbad,San Diego


In [9]:
#extract zips for city
cities_pd = pd.DataFrame(unsorted_df["zip"].unique())
cities_pd



,0
0,91910
1,92126
2,91911
3,92109
4,92101
5,92021
6,92154
7,92104
8,92115
9,92105


In [10]:
cities_pd.to_csv('san_diego_zips.csv', sep=',')


In [11]:
cities_pd = cities_pd.rename(columns = {0: "Zip Code"}) 
cities_pd
len(cities_pd)

87

In [12]:
# Add columns for lat, lng, business name, business rating
# Note that we used "" to specify initial entry.
cities_pd["Lat"] = ""
cities_pd["Lng"] = ""
cities_pd["Florists"] = ""
cities_pd["Bicycle Store"] = ""
cities_pd["Spa"] = ""
cities_pd["Spa Avg Rating"] = ""
cities_pd["Resturaunts Avg Rating"] = ""
cities_pd["Resturaunts"] = ""
cities_pd["Synagogue"] = ""
cities_pd["Veterinary_Care"] = ""
cities_pd["Church"] = ""
cities_pd["Mosque"] = ""
cities_pd["Supermarket"] = ""
cities_pd["Synagogue"] = ""


cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,,,,,,,,,,,,,
1,92126,,,,,,,,,,,,,
2,91911,,,,,,,,,,,,,
3,92109,,,,,,,,,,,,,
4,92101,,,,,,,,,,,,,


In [14]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each zip code
for index, row in cities_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zip_code = row['Zip Code']
   

    # update address key value
    params['address'] = f"San Diego, CA {zip_code}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
#cities_pd.head()

IndexError: list index out of range

In [15]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "florist", 
    #"keyword": "",
    "key": gkey
}


florists = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 5:
            counter +=1
    
    florists.append(counter)
    #row['Florists'] = counter      
            
    #print(florists)x
    
    

In [16]:
cities_pd['Florists'] = florists

In [17]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,,,0,,,,,,,,,,
1,92126,,,0,,,,,,,,,,
2,91911,,,0,,,,,,,,,,
3,92109,,,0,,,,,,,,,,
4,92101,,,0,,,,,,,,,,


In [18]:
#adding bicycle store count

In [19]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "bicycle_store", 
    #"keyword": "",
    "key": gkey
}


bicycle_store = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 5:
            counter +=1
    
    bicycle_store.append(counter)
    #row['Florists'] = counter      
            
    #print(florists)x
    
            
 

In [20]:
cities_pd['Bicycle Store'] = bicycle_store

In [21]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "error_message": "You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account",
  "html_attributions": [],
  "results": [],
  "status": "OVER_QUERY_LIMIT"
}


In [22]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,,,0,0,,,,,,,,,
1,92126,,,0,0,,,,,,,,,
2,91911,,,0,0,,,,,,,,,
3,92109,,,0,0,,,,,,,,,
4,92101,,,0,0,,,,,,,,,


In [26]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "error_message": "You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account",
  "html_attributions": [],
  "results": [],
  "status": "OVER_QUERY_LIMIT"
}


In [24]:
#adding spa count

In [25]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "spa", 
    "keyword": "massage",
    "key": gkey
}


spa = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   


    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    print(cities_response)
    
    
    for coor in cities_response["results"]:
        store_type == coor["types"]
        if len(store_type) <= 5:
            counter +=1
    
    spa.append(counter)

{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active b

{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active b

{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}
{'error_message': 'You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active b

In [24]:
cities_pd['Spa'] = spa 

In [1]:
#checking json to verify data
#print(json.dumps(cities_response, indent=2, sort_keys=True))
cities_response_rating = cities_response.[results][ratings]

SyntaxError: invalid syntax (<ipython-input-1-01ea81bfbc32>, line 3)

In [26]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,,,,,,
1,92126,32.9185,-117.138,3,1,9,,,,,,,,
2,91911,32.606,-117.044,9,3,3,,,,,,,,
3,92109,32.7921,-117.232,10,14,20,,,,,,,,
4,92101,32.727,-117.165,18,9,20,,,,,,,,


In [27]:
#checking json to verify data
#print(json.dumps(cities_response, indent=2, sort_keys=True))

In [28]:
#new dataframe to calculate averages

In [29]:
# add "Spa Avg Rating"

In [30]:
# add restaurant count

In [31]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "restaurant", 
    #"keyword": "",
    "key": gkey
}


restaurant = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 5:
            counter +=1
    
    restaurant.append(counter)

In [32]:
cities_pd['Resturaunts'] = restaurant

In [33]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,17,,,,,
1,92126,32.9185,-117.138,3,1,9,,,17,,,,,
2,91911,32.606,-117.044,9,3,3,,,18,,,,,
3,92109,32.7921,-117.232,10,14,20,,,17,,,,,
4,92101,32.727,-117.165,18,9,20,,,16,,,,,


In [34]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "html_attributions": [],
  "results": [],
  "status": "ZERO_RESULTS"
}


In [35]:
#adding synagogue count

In [36]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "synagogue", 
    #"keyword": "",
    "key": gkey
}


synagogue = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 5:
            counter +=1
  
    synagogue.append(counter)

In [37]:
cities_pd["Synagogue"] = synagogue

In [38]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,17,1,,,,
1,92126,32.9185,-117.138,3,1,9,,,17,0,,,,
2,91911,32.606,-117.044,9,3,3,,,18,0,,,,
3,92109,32.7921,-117.232,10,14,20,,,17,0,,,,
4,92101,32.727,-117.165,18,9,20,,,16,1,,,,


In [39]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))


{
  "html_attributions": [],
  "results": [],
  "status": "ZERO_RESULTS"
}


In [40]:
#adding Veterinary_Care count by zip

In [41]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "veterinary_care", 
    #"keyword": "",
    "key": gkey
}


veterinary_care = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 5:
            counter +=1
    
    veterinary_care.append(counter)

In [42]:
cities_pd['Veterinary_Care'] = veterinary_care

In [43]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,17,1,6,,,
1,92126,32.9185,-117.138,3,1,9,,,17,0,7,,,
2,91911,32.606,-117.044,9,3,3,,,18,0,12,,,
3,92109,32.7921,-117.232,10,14,20,,,17,0,11,,,
4,92101,32.727,-117.165,18,9,20,,,16,1,13,,,


In [44]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "html_attributions": [],
  "results": [
    {
      "geometry": {
        "location": {
          "lat": 33.0490289,
          "lng": -117.1804696
        },
        "viewport": {
          "northeast": {
            "lat": 33.0501421802915,
            "lng": -117.1790517697085
          },
          "southwest": {
            "lat": 33.04744421970851,
            "lng": -117.1817497302915
          }
        }
      },
      "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png",
      "id": "b62f37f12be8387064090ddabc66a10f93cda76e",
      "name": "Sporthorse Veterinary Services",
      "opening_hours": {
        "open_now": false
      },
      "place_id": "ChIJw2DUzd-K24ARvc8Gs2UpeTM",
      "plus_code": {
        "compound_code": "2RX9+JR Rancho Santa Fe, California, United States",
        "global_code": "85542RX9+JR"
      },
      "reference": "ChIJw2DUzd-K24ARvc8Gs2UpeTM",
      "scope": "GOOGLE",
      "types": [
        "veterinary_care",
  

In [45]:
#add church count

In [46]:
# params dictionary to update each iteration
params = {
    "radius": 2000,
    "types": "church", 
    #"keyword": "",
    "key": gkey
}


church = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 5:
            counter +=1
    church.append(counter)

In [47]:
cities_pd['Church'] = church

In [48]:
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,17,1,6,20,,
1,92126,32.9185,-117.138,3,1,9,,,17,0,7,12,,
2,91911,32.606,-117.044,9,3,3,,,18,0,12,13,,
3,92109,32.7921,-117.232,10,14,20,,,17,0,11,20,,
4,92101,32.727,-117.165,18,9,20,,,16,1,13,19,,


In [49]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "html_attributions": [],
  "results": [],
  "status": "ZERO_RESULTS"
}


In [50]:
#adding mosque counts per zip

In [51]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "mosque", 
    #"keyword": "",
    "key": gkey
}


mosque = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 12:
            counter +=1
    
    mosque.append(counter)

In [52]:
cities_pd['Mosque'] = mosque
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,17,1,6,20,0,
1,92126,32.9185,-117.138,3,1,9,,,17,0,7,12,3,
2,91911,32.606,-117.044,9,3,3,,,18,0,12,13,1,
3,92109,32.7921,-117.232,10,14,20,,,17,0,11,20,0,
4,92101,32.727,-117.165,18,9,20,,,16,1,13,19,0,


In [53]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "html_attributions": [],
  "results": [],
  "status": "ZERO_RESULTS"
}


In [54]:
#add supermarket count

In [55]:
# params dictionary to update each iteration
params = {
    "radius": 3000,
    "types": "supermarket", 
    #"keyword": "",
    "key": gkey
}


supermarket = []
# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    counter = 0
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) <= 12:
            counter +=1
    
    supermarket.append(counter)

In [56]:
cities_pd['Supermarket'] = supermarket
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Bicycle Store,Spa,Spa Avg Rating,Resturaunts Avg Rating,Resturaunts,Synagogue,Veterinary_Care,Church,Mosque,Supermarket
0,91910,32.6385,-117.062,9,1,13,,,17,1,6,20,0,6
1,92126,32.9185,-117.138,3,1,9,,,17,0,7,12,3,8
2,91911,32.606,-117.044,9,3,3,,,18,0,12,13,1,9
3,92109,32.7921,-117.232,10,14,20,,,17,0,11,20,0,3
4,92101,32.727,-117.165,18,9,20,,,16,1,13,19,0,7


In [57]:
#checking json to verify data
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "html_attributions": [],
  "results": [],
  "status": "ZERO_RESULTS"
}


In [58]:
# save as csv file

cities_pd.to_csv('sd_biz_zip.csv')